### Bidirectional LSTM
https://towardsdatascience.com/understanding-bidirectional-rnn-in-pytorch-5bd25a5dd66

- 2개의 독립적인 RNN의 결합 (colah's blog 확인)
- 문맥을 고려하기위해 역방향으로도 읽어본

### Word2Vec
- http://jalammar.github.io/illustrated-word2vec/

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt", filename="steam.txt") 

In [ ]:
total_data = pd.read_table('steam.txt', names=['label', 'reviews']) 
print('전체 리뷰 개수 :',len(total_data))

In [ ]:
total_data

In [ ]:
total_data['reviews'].nunique()

In [ ]:
total_data['label'].nunique()

In [ ]:
total_data.drop_duplicates(subset=['reviews'], inplace=True)

In [ ]:
print('총 샘플의 수 :', len(total_data))

In [ ]:
print(total_data.isnull().values.any())

In [ ]:
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

In [ ]:
# 데이터가 한쪽으로 치우칠 경우 맞추어줄 필요가 있다.
train_data['label'].value_counts().plot(kind = 'bar')

In [ ]:
train_data['reviews']

In [ ]:
train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")  #한글이 아닌것을 제거

In [ ]:
train_data['reviews']=train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")  #한글이 아닌것을 제거

In [ ]:
# 공란을 제거할때는 우선 nan으로 처리한 후 제거
train_data['reviews'].replace('', np.nan, inplace=True) 
print(train_data.isnull().sum()) 

In [ ]:
test_data.drop_duplicates(subset = ['reviews'], inplace=True) # 중복 제거
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['reviews'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면'] 

In [ ]:
okt=Okt()

In [ ]:
train_data['tokenized'] = train_data['reviews'].apply(okt.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data['tokenized'] = test_data['reviews'].apply(okt.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords]) 

In [ ]:
negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values) 

In [ ]:
print(np.shape(negative_words))
print(np.shape(positive_words))

In [ ]:
negative_word_count=Counter(negative_words)
negative_word_count.most_common(20)

In [ ]:
positive_word_count=Counter(positive_words)
positive_word_count.most_common(20)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
text_len = train_data[train_data['label']==1]['tokenized'].map(lambda x: len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Positive Reviews')
ax1.set_xlabel('length of samples')
ax1.set_ylabel('number of samples')
print('긍정 리뷰의 평균 길이 :', np.mean(text_len))

text_len = train_data[train_data['label']==0]['tokenized'].map(lambda x: len(x))
ax2.hist(text_len, color='blue')
ax2.set_title('Negative Reviews')
fig.suptitle('Words in texts')
ax2.set_xlabel('length of samples')
ax2.set_ylabel('number of samples')
print('부정 리뷰의 평균 길이 :', np.mean(text_len))
plt.show() 

In [ ]:
X_train = train_data['tokenized'] 
X_train

In [ ]:
y_train = train_data['label'].values 
X_test= test_data['tokenized'].values
y_test = test_data['label'].values 

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) 

In [ ]:
tokenizer.index_word

In [ ]:
y_train

In [ ]:
total_cnt = len(tokenizer.word_index) # 단어의 수

In [ ]:
threshold = 2 
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합 

In [ ]:
# tokenizer.word_counts 
for key, value in tokenizer.word_counts.items(): 
  total_freq = total_freq + value
  # 단어의 등장 빈도수가 threshold보다 작으면 
  if(value < threshold):
    rare_cnt = rare_cnt + 1 # 희귀 단어 카운트
    rare_freq = rare_freq + value #희귀 단어 빈도수 누적

In [ ]:
print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = total_cnt - rare_cnt + 2
# 0 번은 패딩, 1번 oov로 사용, 실제 단어는 2번 index부터 부여
print('단어 집합의 크기 :',vocab_size)  #사용할 단어의 개수

In [ ]:
# Tokenizer(vocab_size) 
#31592를 가지고 토큰화 개체 생성 나머지 단어들은 화면에 출력되지 않는다.
# option이 있다 : OOV token
tokenizer = Tokenizer(vocab_size, oov_token='OOV') #31592개에 포함되지 않는 단어들은 뭘로 할래? 'OOV'로 부탁해~

In [ ]:
tokenizer.fit_on_texts(X_train) # X_train에 대해서 토큰화 진행

In [ ]:
tokenizer.index_word

In [ ]:
tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test) 

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))

In [ ]:
map(len, X_train) # X_train[0]~X_train[마지막] 까지 구해짐

In [ ]:
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

In [ ]:
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  "비율을 구한다."
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
below_threshold_len?
max_len = 60
below_threshold_len(max_len, X_train)

In [ ]:
# 패딩 작업
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
X_train[1]

# 여기까지가 전처리 작업이었다
---

---
# 양방향 LSTM 적용 시작

In [ ]:
import re
from keras.layers import Embedding, Dense, LSTM, Bidirectional
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint 

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100)) # vocab_size (31592) -> 100차원 공간으로 임베딩
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid')) 

In [ ]:
model.summary()

In [ ]:
# es 와 mc 는 따로 작동함
# fitting 할때 early stopping 을 주었음. patience 는 4번
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

In [ ]:
# 값이 계속해서 잘 올라가는 것이 확인 될때 그 시점을 저장
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True) 

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2) 

In [ ]:
# best_model이 코랩 좌측 폴더탭에 생성된것을 확인 후 불러오기
loaded_model = load_model('best_model.h5')

In [ ]:
loaded_model.evaluate(X_test, y_test)

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100)) 

In [ ]:
sentiment_predict('노잼 ..완전 재미 없음 ㅉㅉ')

In [ ]:
sentiment_predict('와 정말 재밌다 좋단다')